# Related Artist Tracks
Now that I can pull all tracks for an artist and I have a simple recommender system that can find similar tracks to the ones I've already liked, I want to crawl through the similar artists to mine and compare their tracks to the ones I've liked to find new, similar tracks to recommend.

Starting with my "Tom's Funky Playlist" I'll get a unique list of artists.

In [1]:
from spoti_pandas import sp_functions as sppd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# # Spotify Credentials
# import spot_creds

# clid = spot_creds.client_id
# secret = spot_creds.secret

# #Authentication - without user
# client_credentials_manager = SpotifyClientCredentials(client_id=clid, client_secret=secret)
# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [2]:
pl_link = '"https://open.spotify.com/playlist/7eWWLoTfmLUcD0viBP6Hr0?si=e8b0760749404749"' # Tom's Funky Playlist
pl_tracks = sppd.pl_track_features(pl_link)
pl_tracks.head()

,track_uri,track_name,artist_id,artist_name,album,track_pop,explicit,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,spotify:track:5ZDLCdSHmpt2JKwWmotjzG,Pants,2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,Single Entendre,0,False,0.02650,0.663,0.851,0.000000,1,0.3580,-4.067,0,0.0351,123.910,4,0.809
1,spotify:track:6NBqMK4DfBYGIzCnmSgliG,Dirty Minds,2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,Everlasting Party,0,False,0.00999,0.603,0.832,0.000000,9,0.0902,-5.705,1,0.0605,132.901,4,0.878
2,spotify:track:4zxXklp6D4vClyC08SUeo9,Carnal Carnival,2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,Carnal Carnival,0,False,0.44300,0.469,0.934,0.000000,11,0.6820,-3.049,0,0.0708,160.061,4,0.753
3,spotify:track:3qaO2Ko1b0bBisqNLU1e1s,Freak Flag,2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,Carnal Carnival,0,False,0.03550,0.651,0.901,0.000000,1,0.3740,-3.393,1,0.0452,109.968,4,0.830
4,spotify:track:6zADCSrbXuHSTnZbznyyY9,Live in the Moment,2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,Live in the Moment,0,False,0.00324,0.693,0.753,0.000215,1,0.1740,-6.065,0,0.0329,105.947,4,0.502


In [3]:
pl_artists = pl_tracks.artist_id.unique()
pl_artists

array(['2pXFmyqPm7wHJ1HGAwyR3L', '7mYdzBqQUa1hb8jdRmMpfE',
       '3afwdwY97SMtIdnUsPTpQT', '5a2EaR3hamoenG9rDuVn8j',
       '5kah03Pzp6s8eD8vRIhdfL', '0DJRN6LYpYlKu30dsXefJv',
       '3U3C9o6UTYNdEsDckpRyvX', '3gfBx0SvMGdMQ2ZsjPvIV4',
       '1wnaeDbP5Yl9MNV9qC008L', '6JmjdmUmKCrzq0CqDt0vB6',
       '4WmMnGO1nLIsE85XwcBAZE', '7JnJgTo8cCtAQmtC0cJyjp',
       '1JyLSGXC3aWzjY6ZdxvIXh', '5rwUYLyUq8gBsVaOUcUxpE',
       '0JCxGVxsISZzJHJPUOtceB', '4GAyr0SBHrn2nL4tvwASuP',
       '6xt9sJmmyYwWkJv8A6ssiU', '7vNcTk9TgKF0qDsS87nWGE',
       '287jMoxHzjERgHI6ja8TKa', '6BufIg68WvvzrEqoDI19Kl',
       '5moJNCJeiNwuQAhDLJXULs', '4RAQZfFgR7NPAWjKwUpEeC'], dtype=object)

## Find Related Artists
For all of the playlist artists, find the related artists and create a unique list.

In [4]:
related_artists = pd.DataFrame()
for artist in pl_artists:
    these_related = sppd.related_artists(artist)
    related_artists = pd.concat([related_artists, these_related])
related_artists.drop_duplicates('name',  inplace=True)

In [5]:
len(related_artists)

324

In [6]:
related_artists.sort_values('popularity', ascending=False, inplace=True)

In [7]:
# artist_uri = '2pXFmyqPm7wHJ1HGAwyR3L' #HCTM
# artist_albums = sppd.get_artist_albums(artist_uri)
# print(f"{len(artist_albums)} albums found.")

# # Test all tracks for artist
# artist_tracks_df = sppd.get_artist_track_features(artist_uri)

## Get all Tracks for Related Artists

In [8]:
start_index = 11

if start_index == 0:
    related_tracks = pd.DataFrame()
else:
    # Pick up where it left off
    related_tracks = pd.read_excel("funky_related_tracks.xlsx")
    related_tracks.drop_duplicates(inplace=True)

len(related_tracks)


1540

In [ ]:
for i in range(len(related_artists[start_index:])):
    artist_index = i + start_index
    artist_name = related_artists.iloc[artist_index]['name']
    artist_uri = related_artists.iloc[artist_index]['uri']
    print(f'{artist_index}/{len(related_artists)} {artist_name}: {artist_uri}')
    
    this_artist_tracks = sppd.get_artist_track_features(artist_uri)
    print(f'{len(this_artist_tracks)} tracks found')
    related_tracks = pd.concat([related_tracks, this_artist_tracks])
    
    #write to excel in case of crash or rate-limit
    related_tracks.drop_duplicates(inplace=True)
    related_tracks.to_excel("funky_related_tracks.xlsx", index=False)
    #sleep to avoid rate limit
    time.sleep(60)

11/324 Curtis Mayfield: spotify:artist:2AV6XDIs32ofIJhkkDevjm
Getting tracks for People Never Give Up
Getting tracks for Keep on Keeping On: Curtis Mayfield Studio Albums 1970-1974 (2019 Remaster)
Sleeping for 60 seconds to avoid rate limit
Getting tracks for The Berlin Sessions
Getting tracks for New World Order
Getting tracks for Take It to the Streets
Getting tracks for Live in Europe
Getting tracks for We Come in Peace With a Message of Love
Getting tracks for Honesty
Getting tracks for Love Is the Place
Getting tracks for Something to Believe In
Getting tracks for The Right Combination (Expanded Edition)
Getting tracks for Heartbeat
Getting tracks for Do It All Night
Getting tracks for Short Eyes - Original Motion Picture Soundtrack
Getting tracks for Never Say You Can't Survive
Getting tracks for Give, Get, Take And Have
Getting tracks for There's No Place Like America Today
Getting tracks for Got to Find a Way
Getting tracks for Sweet Exorcist
Getting tracks for Back to the Worl

HTTP Error for GET to https://api.spotify.com/v1/albums/3x1vnezQHOTQG4kDC8nTzl with Params: {} returned 404 due to Non existing id: 'spotify:album:3x1vnezQHOTQG4kDC8nTzl'


236 tracks found
76/324 Spyro Gyra: spotify:artist:1Be36RHAlqJpfUt3tsmUQD
Getting tracks for Vinyl Tap
Getting tracks for The Rhinebeck Sessions
Getting tracks for Down the Wire
Getting tracks for A Night Before Christmas
Getting tracks for Good to Go-Go
Getting tracks for Wrapped in a Dream
Getting tracks for The Deep End
Getting tracks for Original Cinema
Getting tracks for In Modern Times
Getting tracks for Got The Magic
Getting tracks for Road Scholars
Getting tracks for 20/20
Getting tracks for Heart Of The Night
Getting tracks for Love & Other Obsessions
Getting tracks for Dreams Beyond Control
Getting tracks for Three Wishes
Getting tracks for Fast Forward
Getting tracks for Point Of View
Getting tracks for Rites Of Summer
Getting tracks for Stories Without Words
219 tracks found
77/324 Tab Benoit: spotify:artist:3ODYbknLzVe3cWQzVfbzGB
Getting tracks for Medicine
Getting tracks for Night Train To Nashville (Live)
Getting tracks for Power Of The Pontchartrain
Getting tracks for B

HTTP Error for GET to https://api.spotify.com/v1/albums/6aIMVdJRr57cQ5eY87Ixvz with Params: {} returned 404 due to Non existing id: 'spotify:album:6aIMVdJRr57cQ5eY87Ixvz'


Getting tracks for Cold As Weiss


HTTP Error for GET to https://api.spotify.com/v1/albums/39wAzMaFzTVEN0yqJXYyzK with Params: {} returned 404 due to Non existing id: 'spotify:album:39wAzMaFzTVEN0yqJXYyzK'


Getting tracks for I Told You So


HTTP Error for GET to https://api.spotify.com/v1/albums/1wdTI56LWM9Bcbfx4zVxX4/tracks/ with Params: {'limit': 50, 'offset': 0, 'market': None} returned 404 due to Non existing id: '1wdTI56LWM9Bcbfx4zVxX4'


Getting tracks for Live at KEXP!
Getting tracks for Close but No Cigar
Getting tracks for Fried Soul
Getting tracks for Cold As Weiss
Getting tracks for Jimmy's Groove
Getting tracks for Fo Sho / Inner City Blues
Getting tracks for Concussion
Getting tracks for The Craig Charles Trunk of Funk Volume 2
Getting tracks for Colemine Records Presents: Brighter Days Ahead
Getting tracks for The Craig Charles Funk & Soul Club, Vol. 6
84 tracks found
106/324 David Sanborn: spotify:artist:7vNcTk9TgKF0qDsS87nWGE
Getting tracks for Rocks
Getting tracks for This Masquerade
Sleeping for 60 seconds to avoid rate limit
Getting tracks for Bye Bye Blackbird
Getting tracks for Time and The River
Getting tracks for Enjoy The View
Getting tracks for Quartette Humaine
Getting tracks for Then Again: The David Sanborn Anthology
Sleeping for 60 seconds to avoid rate limit
Getting tracks for Only Everything
Getting tracks for Here & Gone
Getting tracks for Sanborn
Getting tracks for Closer
Getting tracks for T

In [ ]:
len(related_tracks)